In [183]:
import pandas as pd
import psycopg2
from config_postgresql import database, user, password, port, host

In [184]:
connection = psycopg2.connect(database=database, user = user, password=password, port=port, host=host)
connection.autocommit = True

In [185]:
def sql(request):
    cursor = connection.cursor()
    cursor.execute(request)
    result = cursor.fetchall()
    columns = []
    for desc in cursor.description:
        columns.append(desc[0])
    
    return pd.DataFrame(result, columns=columns)

In [6]:
sql('''SELECT 
    CONCAT(module_id, ' ', LEFT(CONCAT(module_name), 14), '...') AS Модуль,
    CONCAT(module_id, '.', lesson_position, ' ', LEFT(CONCAT(lesson_name), 12), '...') AS Урок,
    CONCAT(module_id, '.', lesson_position, '.', step_position, step_name) AS Шаг
FROM
    module
    INNER JOIN lesson USING(module_id)
    INNER JOIN step USING(lesson_id)
WHERE step_name LIKE '%вложен%запрос%'
ORDER BY Модуль, Урок, Шаг;''')

,Модуль,Урок,Шаг
0,1 Основы реляцио...,1.4 Вложенные за...,1.4.3Использование вложенного запроса в выражении
1,1 Основы реляцио...,1.4 Вложенные за...,1.4.3Использование вложенного запроса в выражении
2,1 Основы реляцио...,1.5 Запросы корр...,"1.5.5Добавление записей, вложенные запросы"
3,1 Основы реляцио...,1.5 Запросы корр...,"1.5.5Добавление записей, вложенные запросы"
4,2 Запросы SQL к ...,2.2 Запросы на в...,2.2.7Запросы для нескольких таблиц со вложенны...
5,2 Запросы SQL к ...,2.2 Запросы на в...,2.2.7Запросы для нескольких таблиц со вложенны...
6,2 Запросы SQL к ...,2.3 Запросы корр...,"2.3.5Запрос на обновление, вложенные запросы"
7,2 Запросы SQL к ...,2.3 Запросы корр...,"2.3.5Запрос на обновление, вложенные запросы"


Вывести первых 10 `student_id` с самым большим числом правильных ответов.

In [89]:
sql('''select student_id, count(result) as Количество
from step_student
where result = 'correct'
group by student_id
order by Количество desc
limit 10''')

,student_id,Количество
0,53,68
1,60,66
2,36,62
3,9,62
4,46,62
5,30,62
6,39,62
7,15,62
8,59,60
9,4,60


In [138]:
sql('''


select student_name, sum((submission_time - attempt_time)) as sec from step_student

join student using(student_id)

where result = 'correct'

group by student_name

order by sec desc

''')

,student_name,sec
0,student_31,1916275
1,student_4,1087469
2,student_57,995521
3,student_43,688631
4,student_45,668937
...,...,...
59,student_18,1079
60,student_64,924
61,student_3,897
62,student_7,535


2. Выведите топ-20 студентов по объему освоенного курса, добавить столбцы - % пройденного материала, среднее время решения задания, количество неправильных попыток.

In [223]:
sql('''
with correct_raiting (student_id, count_step, avg_sec) as
    (
        select student_id, count(distinct step_id) as count_step,
        round(avg(submission_time - attempt_time)) as avg_sec
        from step_student
        where result = 'correct'
        group by student_id
    ),
wrong_raiting (student_id, count_wrong) as
    (
        select student_id, count(result) as count_wrong
        from step_student
        where result = 'wrong'
        group by student_id    
    )
select student_name, 
(count_step*100/(select count(distinct step_id) from step_student)) as percent, count_wrong, avg_sec 
from correct_raiting
    join wrong_raiting using(student_id)
    join student using(student_id)
order by percent desc
limit 20
''')

,student_name,percent,count_wrong,avg_sec
0,student_60,96,9,1984
1,student_43,93,140,22954
2,student_30,93,1,19014
3,student_53,93,13,3732
4,student_46,93,2,5771
5,student_31,93,3,63876
6,student_27,93,50,6935
7,student_18,93,11,36
8,student_36,93,105,4299
9,student_39,93,9,747
